# Question 5

Resource: https://web.itu.edu.tr/topcuil/ya/SEN301previousexamquestions.pdf

Maçka Police Station employs 30 police officers. Each officer works for 5 days per week. The crime rate fluctuates with the day of week, so the number of the police officers required each day depends on which day of the week it is: Monday, 18; Tuesday, 24; Wednesday, 25; Thursday, 16; Friday, 21; Saturday, 28; Sunday, 18. The Police Station wants to schedule police officers to minimize the number whose days off are not consecutive. Formulate an LP that will accomplish this goal. 

In [31]:
# Since there are 7 days in a week, and the minimum work days is 5, the ideal case is:
# work 2 days, take 1 day off, work 2 days again, take 1 day off, work 1 day.

In [40]:
from ortools.sat.python import cp_model

n_officers = 30
n_days = 5
officers_per_day = [18, 24, 25, 16, 21, 28, 18]

model = cp_model.CpModel()

work_days = {}
for i in range(n_officers):
  for j in range(7):
    work_days[i, j] = model.NewBoolVar( f'officer{i}-day{j}')
  
  # Each officer works 5 days per week.
  model.Add(sum(work_days[i, j] for j in range(7)) == n_days)

# Each day has a fixed number of officers.
for i in range(7):
  model.Add(sum(work_days[j, i] for j in range(n_officers)) == officers_per_day[i])

# https://github.com/google/or-tools/blob/stable/ortools/sat/docs/boolean_logic.md#product-of-two-boolean-variables
# Track how often each officer take days off on consecutive days.
# For every 3 days, if the officer takes 2 days off, then the officer only works for 1 day.
for i in range(n_officers):
  for j in range(7):
    # We want them to work consecutively.
    model.Add(sum(work_days[i, (j+k) % 7] for k in range(3)) >= 2)

solver = cp_model.CpSolver()
status = solver.Solve(model)
solver.StatusName(status)


'OPTIMAL'

In [42]:
from collections import defaultdict
officers_by_day = defaultdict(list)
days_by_officer = defaultdict(list)

for i in range(n_officers):
  for j in range(7):
    if solver.Value(work_days[i, j]):
      officers_by_day[j].append(i)
      days_by_officer[i].append(j)

for j in range(7):
  print(f'Day {j}: {len(officers_by_day[j])} {officers_by_day[j]}')

Day 0: 18 [0, 1, 5, 6, 9, 10, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Day 1: 24 [2, 3, 4, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Day 2: 25 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Day 3: 16 [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 12, 25, 26, 27, 28, 29]
Day 4: 21 [0, 3, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Day 5: 28 [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Day 6: 18 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
